<a href="https://colab.research.google.com/github/axel-sirota/practical-nlp/blob/main/3-text-generation/Practical_NLP_9_RentalGenerator_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install textblob 'gensim==4.2.0' 'keras-nlp'

In [ ]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
corpus_size=25000
test_corpus_size=5000
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f train_corpus_descriptions_airbnb.csv ]; then
  wget -O train_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
fi

if [ ! -f test_corpus_descriptions_airbnb.csv ]; then
    wget -O test_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/a29bbkg8hi4q4f4/test_corpus_descriptions_airbnb.csv?dl=0
fi

In [ ]:
!bash get_data.sh

In [ ]:
train_path = "./train_corpus_descriptions_airbnb.csv"
test_path = "./test_corpus_descriptions_airbnb.csv"
# Read, then decode for py2 compat.
airbnb_reviews = pd.read_csv(train_path, header=None, names=["review"]).dropna().sample(n=corpus_size).reset_index(drop=True)
test_airbnb_reviews = pd.read_csv(test_path, header=None, names=["review"]).dropna().sample(n=test_corpus_size).reset_index(drop=True)


In [ ]:
airbnb_reviews

In [ ]:
# Take a look at the first review in text
print(airbnb_reviews.iloc[0].review)


In [ ]:
def preprocess_text(text, should_join=True):
    text = str(text)
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [ ]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(preprocess_text(row.review, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(airbnb_reviews)


In [ ]:
vocab = set()
counts = defaultdict(int)
for ix, review in airbnb_reviews.iterrows():
    for word in preprocess_text(review['review'], should_join=False):
        counts[word] += 1
        if counts[word] > min_count_words:
          vocab.add(word)
vocab = sorted(vocab)


In [ ]:
print(f'{len(vocab)} unique words')


In [ ]:
ids_from_words = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [ ]:
words_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

In [ ]:
ids = ids_from_words(preprocess_text('Only you can prevent forest fires', should_join=False))
ids

In [ ]:
text_from_ids(ids)

In [ ]:
def get_ids_tensor(df):
  all_ids = tf.constant(np.zeros((1, maximum)), dtype='int64')
  for review in df.review:
      review = preprocess_text(review, should_join=False)
      value = ids_from_words(review[0: maximum])
      if len(review) < maximum:
        value = tf.concat([value, tf.zeros((maximum - len(review)), dtype='int64')], axis=0) 
      value = tf.reshape(value, [1, maximum])
      output = tf.concat([all_ids,value], axis=0)
      all_ids = tf.reshape(output, [-1, maximum])
  return all_ids[1:]

all_ids = get_ids_tensor(df=airbnb_reviews)
print(all_ids)

In [ ]:
test_all_ids = get_ids_tensor(df=test_airbnb_reviews)

In [ ]:
#Prepare the dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_all_ids)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = ids_dataset.map(split_input_target)
test_dataset = test_ids_dataset.map(split_input_target)


In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size=batch_size, drop_remainder=True)
)
test_dataset = (
    test_dataset
    .batch(batch_size=batch_size, drop_remainder=True)
)

In [ ]:
dataset.take(1)

In [ ]:
class RentalGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn = tf.keras.layers.LSTM(rnn_units,
                                   activation='tanh',
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.rnn.get_initial_state(x)
    output = self.rnn(x, initial_state=states, training=training)
    x = output[0]
    states = output[1:]
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = RentalGenerator(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_words.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[perplexity])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, min_delta=0.03)
history = model.fit(dataset, epochs=epochs, validation_data=test_dataset, workers=5, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], val_metric=history.history['val_loss'], metric_name="Loss", title="Loss", ylim=5.0)

In [ ]:
plot_metrics(history.history['perplexity'], val_metric=history.history['val_perplexity'], metric_name="perplexity", title="perplexity", ylim=2000.0)

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, word_from_ids, ids_from_words, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.word_from_ids = word_from_ids
    self.ids_from_words = ids_from_words

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_words(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_words.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function(reduce_retracing=True)
  def expand_dims_if_neccesary(self, input):
    if len(input.shape) < 3:
      input = tf.expand_dims(input, axis=0)
    return input

  @tf.function(reduce_retracing=True)
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_words = preprocess_text(inputs, should_join=False)
    input_ids = self.expand_dims_if_neccesary(self.ids_from_words(input_words))
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_word = self.word_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_word, states

In [ ]:
one_step_model = OneStep(model, words_from_ids, ids_from_words)
start = time.time()
states = None
description = tf.constant(['Midtown Sunny 2-Bedroom'])
# result = [description]

for n in range(100):
  next_word, states = one_step_model.generate_one_step(description, states=states)
  description = tf.concat([description, next_word], axis=0)


result = tf.strings.join(description, separator=" ")
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)



In [ ]:
tf.saved_model.save(one_step_model, 'lstm_rental_generator')